In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import soundfile as sf
import os
from tqdm.notebook import tqdm
!pip install pydub
from pydub import AudioSegment
import IPython
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [54]:
one_annotation_file = pd.read_csv("/content/drive/MyDrive/mosaic/text_filess/103_2b2_Ar_mc_LittC2SE.txt", sep = '\t')

In [55]:
one_annotation_file

,0.364,3.25,0,1
0,3.250,6.636,0,0
1,6.636,11.179,0,1
2,11.179,14.250,0,1
3,14.250,16.993,0,1
4,16.993,19.979,0,0


In [58]:
directory = "/content/drive/MyDrive/mosaic/text_files"
patient_id = []
recording_idx = []
chest_location = []
acquisition_mode = []
recording_equipment = []

for file in os.listdir(directory):
  if file.endswith('.txt'):
    file = file.split('_')
    patient_id.append(file[0])
    recording_idx.append(file[1])
    chest_location.append(file[2])
    acquisition_mode.append(file[3])
    recording_equipment.append(file[4])


data = {
    'patient_id' : patient_id,
    'recording_idx' : recording_idx,
    'chest_location' : chest_location,
    'acquisition_mode' : acquisition_mode,
    'recording_equipment' : recording_equipment
}

annotation_df = pd.DataFrame(data)
annotation_df.head()

,patient_id,recording_idx,chest_location,acquisition_mode,recording_equipment
0,172,1b5,Al,mc,AKGC417L.txt
1,220,1b1,Tc,mc,LittC2SE.txt
2,186,2b2,Tc,mc,AKGC417L.txt
3,179,1b1,Al,sc,Meditron.txt
4,211,1p2,Pr,mc,AKGC417L.txt


In [59]:
annotation_df.shape

(810, 5)

In [61]:
def get_file_name(file):
    return file.split('_')

In [65]:
import os
file_data = []
for file in os.listdir(directory):
  if file.endswith('.txt'):
    data = pd.read_csv(directory + '/' + file , sep='\t', names = ['start','end','crackles','wheezes'])
    filename = file.split('.')
    filename_without_extension = filename[0]
    individual_file = get_file_name(filename_without_extension)
    data['patient_id'] = individual_file[0]
    data['recording_idx'] = individual_file[1]
    data['chest_location'] = individual_file[2]
    data['acquisition_mode'] = individual_file[3]
    data['filename'] = filename_without_extension
    file_data.append(data)

In [66]:
file_data

[    start     end  crackles  wheezes patient_id recording_idx chest_location  \
 0   1.010   3.581         0        0        172           1b5             Al   
 1   3.581   6.836         0        1        172           1b5             Al   
 2   6.836   9.624         1        0        172           1b5             Al   
 3   9.624  12.262         0        0        172           1b5             Al   
 4  12.262  15.801         0        0        172           1b5             Al   
 5  15.801  18.322         0        0        172           1b5             Al   
 6  18.322  19.274         0        0        172           1b5             Al   
 
   acquisition_mode                filename  
 0               mc  172_1b5_Al_mc_AKGC417L  
 1               mc  172_1b5_Al_mc_AKGC417L  
 2               mc  172_1b5_Al_mc_AKGC417L  
 3               mc  172_1b5_Al_mc_AKGC417L  
 4               mc  172_1b5_Al_mc_AKGC417L  
 5               mc  172_1b5_Al_mc_AKGC417L  
 6               mc  172_1b5

In [67]:
files_df = pd.concat(file_data)
files_df.reset_index()
files_df.head()

,start,end,crackles,wheezes,patient_id,recording_idx,chest_location,acquisition_mode,filename
0,1.010,3.581,0,0,172,1b5,Al,mc,172_1b5_Al_mc_AKGC417L
1,3.581,6.836,0,1,172,1b5,Al,mc,172_1b5_Al_mc_AKGC417L
2,6.836,9.624,1,0,172,1b5,Al,mc,172_1b5_Al_mc_AKGC417L
3,9.624,12.262,0,0,172,1b5,Al,mc,172_1b5_Al_mc_AKGC417L
4,12.262,15.801,0,0,172,1b5,Al,mc,172_1b5_Al_mc_AKGC417L


In [68]:
files_df.shape

(6110, 9)

In [69]:
files_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6110 entries, 0 to 5
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   start             6110 non-null   float64
 1   end               6110 non-null   float64
 2   crackles          6110 non-null   int64  
 3   wheezes           6110 non-null   int64  
 4   patient_id        6110 non-null   object 
 5   recording_idx     6110 non-null   object 
 6   chest_location    6110 non-null   object 
 7   acquisition_mode  6110 non-null   object 
 8   filename          6110 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 477.3+ KB


In [70]:
files_df['crackles'].value_counts()

crackles
0    3936
1    2174
Name: count, dtype: int64

In [71]:
files_df['wheezes'].value_counts()

wheezes
0    4880
1    1230
Name: count, dtype: int64

In [72]:
files_df.to_csv('/content/drive/MyDrive/New_folder/data')